# Match do emprego

In [17]:
import pandas as pd
import numpy as np

In [10]:
# df_vagas = pd.read_csv('data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_VAGAS_PR.csv', encoding = "ISO-8859-1",sep=';')
df_vagas = pd.read_excel('data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_VAGAS_PR.xlsx')
pd.set_option('max_colwidth', None)
df_vagas.head()

,CODIGO_OCUPACAO,TITULO_OCUPACAO,OBSERVACOES_OCUPACAO,CODIGO_MUNICIPIO_IBGE,CEP,VALOR_SALARIO,QTD_VAGAS,TIPO_CONTRATACAO,QTD_EXPERIENCIA,ESCOLARIDADE,REQUERIDO_DEFICIENCIA,DEFICIENCIAS
0,514120.0,Zelador,NaN,410840.0,85601000.0,600.0,2.0,3 - Permanente,2.0,10 - Fundamental Incompleto,0.0,11 - Parcial - Física
1,783215.0,Carregador e descarregador de caminhões,para trabalhar em empresa com cargas de farinha - area industrial do morumbi,410830.0,85858700.0,812.0,100.0,3 - Permanente,3.0,10 - Fundamental Incompleto,0.0,11 - Parcial - Física
2,142205.0,Coordenador de rh,"ASSESSORAR OS CLIENTES DA EMPRESA NA GESTÃO DA ÁREA DE RECURSOS HUMANOS, MAPEANDO OS PROCESSOS EXISTENTES E PROCURANDO A MELHORIA CONTINUA DAS ATIVIDADES. DISPONIBILIDADE TOTAL PARA VIAGENS.",411520.0,87015280.0,0.0,1.0,3 - Permanente,6.0,Superior Completo,0.0,2 - Amputação - Física|5 - Um membro Inferior - Física|7 - Um membro superior - Física
3,513205.0,Cozinheiro geral,"Experiência anterior em cozinha, noções de Boas práticas de manipulação de alimentos",410830.0,85852000.0,1200.0,1.0,3 - Permanente,6.0,5 - Fundamental Completo,0.0,5 - Um membro Inferior - Física|8 - Nanismo - Física
4,763215.0,"Costureiro, a máquina na confecção em série",01 para interloque; 01 para overloque; 01 para presponto e coluna. com experiência.,410140.0,86802260.0,1656.0,3.0,3 - Permanente,6.0,0 - Nenhum,0.0,0 - Surdez bilateral parcial - Auditiva|1 - Surdez bilateral total - Auditiva|2 - Amputação - Física|3 - Cadeirante - Física|4 - Membros Inferiores - Física|5 - Um membro Inferior - Física|6 - Membros Superiores - Física|7 - Um membro superior - Física|8 - Nanismo - Física|9 - Ostomia - Física|10 - Paralisia Cerebral - Física|11 - Parcial - Física|12 - Mental / Intelectual - Mental| 13 - Baixa Visão - Visual|14 - Cegueira - Visual


### Pré-processamento vagas 
- tipo_contratacao e escolaridade: substituindo valores textuais para numéricos
- qtd_experiencia: preenchendo campos vazios com zero e convertendo campo float para int**

In [14]:
# Remove as colunas que não serão utilizadas
df_vagas=df_vagas.drop(['REQUERIDO_DEFICIENCIA', 'DEFICIENCIAS','CEP','OBSERVACOES_OCUPACAO','TITULO_OCUPACAO','QTD_VAGAS','CODIGO_MUNICIPIO_IBGE'], axis=1)

df_vagas.rename(columns={'CODIGO_OCUPACAO': 'cod_ocupacao', 'VALOR_SALARIO': 'salario', 'TIPO_CONTRATACAO': 'tipo_contratacao',
'QTD_EXPERIENCIA': 'qtd_experiencia', 'ESCOLARIDADE': 'escolaridade'}, inplace=True)

In [18]:

# Tipo contratacao
df_vagas.loc[df_vagas['tipo_contratacao'] == '0 - Estágio', 'tipo_contratacao']     = 0
df_vagas.loc[df_vagas['tipo_contratacao'] == '1 - Aprendiz', 'tipo_contratacao']    = 1 
df_vagas.loc[df_vagas['tipo_contratacao'] == '2 - Temporária', 'tipo_contratacao']  = 2
df_vagas.loc[df_vagas['tipo_contratacao'] == '3 - Permanente', 'tipo_contratacao']  = 3

# Escolaridade
df_vagas.loc[df_vagas['escolaridade'] == '0 - Nenhum', 'escolaridade']  = 0
df_vagas.loc[df_vagas['escolaridade'] == '10 - Fundamental Incompleto', 'escolaridade']  = 1
df_vagas.loc[df_vagas['escolaridade'] == '5 - Fundamental Completo', 'escolaridade']  = 2
df_vagas.loc[df_vagas['escolaridade'] == '6 - Médio Incompleto', 'escolaridade']  = 3
df_vagas.loc[df_vagas['escolaridade'] == '7 - Médio Completo', 'escolaridade']  = 4
df_vagas.loc[df_vagas['escolaridade'] == '8 - Superior Incompleto', 'escolaridade']  = 5
df_vagas.loc[df_vagas['escolaridade'] == 'Superior Completo', 'escolaridade']  = 6
df_vagas.loc[df_vagas['escolaridade'] == '13 - Especialização', 'escolaridade']  = 7

# Qtd. Experiência
df_vagas.fillna(0, inplace=True)
df_vagas.qtd_experiencia =df_vagas.qtd_experiencia.astype(int)

df_vagas['cod_ocupacao'] = df_vagas.cod_ocupacao.astype(np.int64)
df_vagas['salario'] = df_vagas.salario.astype(np.int64)

df_vagas.to_excel('data/vagas-pr.xlsx')
df_vagas

,cod_ocupacao,salario,tipo_contratacao,qtd_experiencia,escolaridade
0,514120,600,3,2,1
1,783215,812,3,3,1
2,142205,0,3,6,6
3,513205,1200,3,6,2
4,763215,1656,3,6,0
...,...,...,...,...,...
4041,623110,1600,3,0,0
4042,784205,1212,3,1,0
4043,784205,1700,3,0,4
4044,422120,1366,3,0,0


### Pré-processamento candidatos
- deficiencia: 0 (não), 1 (sim)
- escolaridade: domínio de -1 (Não Identificado) a 8 (Doutorado)

In [15]:
df_candidatos = pd.read_excel('data/candidatos-pr.xlsx')
df_candidatos.head() #2m 53.6s

KeyboardInterrupt: 

In [12]:
import re 

def getOcupacoes(txtOcupacoes):
    pattern_cod_ocupacao = '[\d]{6}' 

    ocupacoes = txtOcupacoes.split('|')
    cod_ocupacao = []

    for partOcupacao in ocupacoes:
        print('Ocupação pretendida: ' + partOcupacao)
        cod_ocupacao += re.findall(pattern_cod_ocupacao, partOcupacao)

    return cod_ocupacao


In [14]:
# Deficiência
df_candidatos.loc[df_candidatos['deficiencia'] == '', 'deficiencia'] = 0
df_candidatos.loc[df_candidatos['deficiencia'] != '', 'deficiencia'] = 1

# Escolaridade
df_candidatos.loc[df_candidatos['escolaridade'] == 'Não Identificado', 'escolaridade']  = -1
df_candidatos.loc[df_candidatos['escolaridade'] == 'Nenhum', 'escolaridade']  = 0
df_candidatos.loc[df_candidatos['escolaridade'] == 'Analfabeto', 'escolaridade']  = 0
df_candidatos.loc[df_candidatos['escolaridade'] == 'Fundamental Incompleto', 'escolaridade']  = 1
df_candidatos.loc[df_candidatos['escolaridade'] == 'Fundamental Completo', 'escolaridade']  = 2
df_candidatos.loc[df_candidatos['escolaridade'] == 'Médio Incompleto', 'escolaridade']  = 3
df_candidatos.loc[df_candidatos['escolaridade'] == 'Médio Completo', 'escolaridade']  = 4
df_candidatos.loc[df_candidatos['escolaridade'] == 'Superior Incompleto', 'escolaridade']  = 4
df_candidatos.loc[df_candidatos['escolaridade'] == 'Superior Completo', 'escolaridade']  = 5
df_candidatos.loc[df_candidatos['escolaridade'] == 'Especialização', 'escolaridade']  = 6
df_candidatos.loc[df_candidatos['escolaridade'] == 'Mestrado', 'escolaridade']  = 7
df_candidatos.loc[df_candidatos['escolaridade'] == 'Doutorado', 'escolaridade']  = 8

# Estudante
df_candidatos.loc[df_candidatos['estudante'] == 'N', 'estudante'] = 0
df_candidatos.loc[df_candidatos['estudante'] == 'S', 'deficiencia'] = 1


# Habilitacao
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()

habilitacao_X = df_candidatos.loc[:, 'habilitacao'] # column habilitacao
df_candidatos.loc[:, 'habilitacao'] = labelencoder_X.fit_transform(habilitacao_X)

# Veículo
df_candidatos.loc[df_candidatos['veiculo'] == 'N', 'veiculo'] = 0
df_candidatos.loc[df_candidatos['veiculo'] == 'S', 'veiculo'] = 1

# Disponibilidade viajar
df_candidatos.loc[df_candidatos['disp_viajar'] == 'N', 'disp_viajar'] = 0
df_candidatos.loc[df_candidatos['disp_viajar'] == 'S', 'disp_viajar'] = 1

# Disponibilidade dormir
df_candidatos.loc[df_candidatos['disp_dormir'] == 'N', 'disp_dormir'] = 0
df_candidatos.loc[df_candidatos['disp_dormir'] == 'S', 'disp_dormir'] = 1

# Disponibilidade ausentar domicilio
df_candidatos.loc[df_candidatos['disp_ausentar_domicilio'] == 'N', 'disp_ausentar_domicilio'] = 0
df_candidatos.loc[df_candidatos['disp_ausentar_domicilio'] == 'S', 'disp_ausentar_domicilio'] = 1

# Ocupacao pretendida
# retirando linhas cuja ocupacao_pretendida não tem valor
# df_candidatos = df_candidatos[df_candidatos['ocupacao_pretendida'].notna()]
df_candidatos = df_candidatos[df_candidatos.ocupacao_pretendida != '']

df_candidatos.count()

for index, row in df_candidatos.iterrows():    
    df_candidatos.at[index, 'ocupacao_pretendida'] = getOcupacoes(row['ocupacao_pretendida'].strip())


# df_candidatos.to_excel('data/candidatos-pr.xlsx')
# df_candidatos

AttributeError: 'list' object has no attribute 'strip'